# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [77]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [78]:
# Part 1 data file
cities_metadata_path = "../WeatherPy/city_data.csv"

# Read the metadata 
cities_metadata_df = pd.read_csv(cities_metadata_path)

In [109]:
cities_metadata_df.columns

Index(['City', 'Lat', 'Long', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Time'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [81]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [82]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_metadata_df[["Lat", "Long"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
location_hum = cities_metadata_df.dropna()
loc_humidity = cities_metadata_df["Humidity"].astype(float)


In [102]:
# Create a humidity heatmap layer
fig = gmaps.figure(map_type='HYBRID', center = (0, 30) ,zoom_level = 2)

humidity_layer = gmaps.heatmap_layer(locations, weights=loc_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [106]:
cities_metadata_df.columns()

TypeError: 'Index' object is not callable

In [137]:
# Narrow down the DataFrame to find your ideal weather condition

#A max temperature lower than 30 degrees but higher than 25.
#Wind speed less than 4.5 m/s.
#Zero cloudiness.

ideal_max_temp = 33
ideal_min_temp = 24
ideal_wind_speed = 4.5
ideal_cloudiness = 0 

ideal_location_df = cities_metadata_df[(cities_metadata_df["Max Temp"] <= ideal_max_temp) &
                                        (cities_metadata_df["Max Temp"] >= ideal_min_temp) &
                                        (cities_metadata_df["Wind Speed"] <= ideal_wind_speed) &
                                        (cities_metadata_df["Cloudiness"] == ideal_cloudiness)] 


#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

ideal_location_df.head(20)


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Time
0,Ponta do Sol,32.6667,-17.1000,25.40,74,0,1.37,PT,<module 'time' (built-in)>
22,Bengkulu,-3.8004,102.2655,24.45,84,0,1.89,ID,<module 'time' (built-in)>
34,Saint George,37.1041,-113.5841,27.66,54,0,3.09,US,<module 'time' (built-in)>
111,San Angelo,31.4638,-100.4370,30.53,50,0,3.60,US,<module 'time' (built-in)>
204,Nejapa,13.8138,-89.2306,26.78,92,0,2.06,SV,<module 'time' (built-in)>
222,Russell,32.3502,-85.1999,31.09,74,0,0.00,US,<module 'time' (built-in)>
258,Sussex,41.1834,-74.6663,30.60,73,0,2.06,US,<module 'time' (built-in)>
394,‘Ewa Beach,21.3156,-158.0072,24.22,81,0,2.57,US,<module 'time' (built-in)>
458,Vilhena,-12.7406,-60.1458,26.12,65,0,2.06,BR,<module 'time' (built-in)>
471,Dvinskoy Bereznik,62.8551,42.7072,27.48,46,0,2.68,RU,<module 'time' (built-in)>


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [256]:
# Store into variable named hotel_df

hotel_search_df = ideal_location_df[["City", "Country", "Lat", "Long"]]

hotel_search_df

,City,Country,Lat,Long
0,Ponta do Sol,PT,32.6667,-17.1000
22,Bengkulu,ID,-3.8004,102.2655
34,Saint George,US,37.1041,-113.5841
111,San Angelo,US,31.4638,-100.4370
204,Nejapa,SV,13.8138,-89.2306
222,Russell,US,32.3502,-85.1999
258,Sussex,US,41.1834,-74.6663
394,‘Ewa Beach,US,21.3156,-158.0072
458,Vilhena,BR,-12.7406,-60.1458
471,Dvinskoy Bereznik,RU,62.8551,42.7072


In [259]:
# set up additional columns to hold information
hotel_search_df['Hotel'] = ""
hotel_search_df.head(20)

C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Long,Hotel
0,Ponta do Sol,PT,32.6667,-17.1000,
22,Bengkulu,ID,-3.8004,102.2655,
34,Saint George,US,37.1041,-113.5841,
111,San Angelo,US,31.4638,-100.4370,
204,Nejapa,SV,13.8138,-89.2306,
222,Russell,US,32.3502,-85.1999,
258,Sussex,US,41.1834,-74.6663,
394,‘Ewa Beach,US,21.3156,-158.0072,
458,Vilhena,BR,-12.7406,-60.1458,
471,Dvinskoy Bereznik,RU,62.8551,42.7072,


In [260]:
hotel_search_df.dtypes

City        object
Country     object
Lat        float64
Long       float64
Hotel       object
dtype: object

In [268]:
# Set the hotel search radius

search_type = "lodging"
search_radius = 5000


search_loc = "37.1041,-113.5841" # Saint George

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": search_loc,
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print(f"A Hotel at {search_loc}")

response = requests.get(base_url, params)

search_result = response.json()

# Print the name and address of the first hotel to appear
print(search_result["results"][0]["name"])

print(search_result["results"][0]["rating"])
#print(params)

A Hotel at 37.1041,-113.5841
Best Western Plus Abbey Inn
4.6


In [267]:
# Set the hotel search radius

search_type = "lodging"
search_radius = 5000

search_loc = "" # Saint George


params = {
    #"location": search_loc,
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_search_df.iterrows():

    
    # get location from df
    search_city = row['City']
    
    print(search_city)
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    
    print(lat)
    print(lng)
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    
    params["location"] = search_loc
    
    #params
    
    print(search_loc)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    
    print(f"A Hotel at {search_city}")

    response = requests.get(base_url, params=params)
    
    
    
    search_result = response.json()
    
    #try:
    # Print the name and address of the first hotel to appear
    print(f"A hotel in {search_loc} to try is {search_result[0]['name']}.")
        
    #except (KeyError, IndexError):
    #print("No results... skipping.")
        
    print("----------------------------")

Ponta do Sol
32.6667
-17.1
32.6667,-17.1
A Hotel at Ponta do Sol


KeyError: 0

In [270]:
# Set the hotel search radius

search_type = "lodging"
search_radius = 5000

search_loc = "" # Saint George


params = {
    #"location": search_loc,
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_search_df.iterrows():

    
    # get location from df
    search_city = row['City']
    
    print(search_city)
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    
    print(lat)
    print(lng)
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    
    params["location"] = search_loc
    
    #params
    
    print(search_loc)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    
    print(f"A Hotel at {search_city}")

    response = requests.get(base_url, params=params)
    
    
    
    search_result = response.json()
    
    try:
        # Print the name and address of the first hotel to appear
        print(f"A hotel in {search_city} to try is {search_result[0]['name']}.")
        
    except (KeyError, IndexError):
        print("No results... skipping.")
        
    print("----------------------------")

Ponta do Sol
32.6667
-17.1
32.6667,-17.1
A Hotel at Ponta do Sol
No results... skipping.
----------------------------
Bengkulu
-3.8004
102.2655
-3.8004,102.2655
A Hotel at Bengkulu
No results... skipping.
----------------------------
Saint George
37.1041
-113.5841
37.1041,-113.5841
A Hotel at Saint George
No results... skipping.
----------------------------
San Angelo
31.4638
-100.43700000000001
31.4638,-100.43700000000001
A Hotel at San Angelo
No results... skipping.
----------------------------
Nejapa
13.8138
-89.2306
13.8138,-89.2306
A Hotel at Nejapa
No results... skipping.
----------------------------
Russell
32.3502
-85.1999
32.3502,-85.1999
A Hotel at Russell
No results... skipping.
----------------------------
Sussex
41.1834
-74.6663
41.1834,-74.6663
A Hotel at Sussex
No results... skipping.
----------------------------
‘Ewa Beach
21.3156
-158.0072
21.3156,-158.0072
A Hotel at ‘Ewa Beach
No results... skipping.
----------------------------
Vilhena
-12.7406
-60.1458
-12.7406,-60

In [202]:
sc = f"{search_coords}"
sc

'(62.8551, 42.7072)'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

# Create hotel (ideal symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)

# Display figure
fig = gmaps.figure()
fig.add_layer(bank_layer)

fig